In [ ]:
import re 
import sys 
import json
from io import StringIO
from abc import ABC, abstractmethod
from enum import Enum
from typing import Dict, List, Any, Optional
from dataclasses import dataclass

from langchain_ollama import (
  OllamaLLM,
  ChatOllama,
  OllamaEmbeddings
)
from langchain_core.messages import (
  BaseMessage,
  AIMessage,
  HumanMessage  
)
from langchain_core.prompts import (
  ChatPromptTemplate,
  MessagesPlaceholder  
)


GEMMA = "gemma3:1b"             # ollama gemma3:1b
DEEPSEEK = "deepseek-r1:1.5b"   # ollama deepseek-r1:1.5b

In [ ]:
DEFAULT_SYSTEM_PROMPT = """ 
Eres un asistente especializado capaz de responder preguntas sencillas
"""

In [ ]:
class LLM(ABC):
  def __init__(self, model:str, api_key:str="", temperature:float=0.7):
    self.model = model
    self.api_key = api_key 
    self.temperature = temperature
  
  @abstractmethod
  def __call__(self, query:str) -> str:
    pass 

class Ollama(LLM):
  def __init__(self, model:str, api_key:str = "", temperature:float = 0.7, system_prompt:str = ""):
    super().__init__(model, api_key, temperature)
    
    try:
      self.llm = OllamaLLM(self.model, self.temperature)
    except Exception:
      self.llm = OllamaLLM(model=GEMMA, temperature=0.7)
    
    self.system_prompt:str = system_prompt if len(system_prompt) > 0 else DEFAULT_SYSTEM_PROMPT
    self.memory:List = []
    
    self.chat_prompt = ChatPromptTemplate.from_messages(
      [
        ('system', f'{self.system_prompt}'),
        MessagesPlaceholder(variable_name='memory'),
        ('human', '{query}')
      ]
    )
    self.chain = self.chat_prompt | self.llm  
  
  def __call__(self, query:str) -> str:
    response = self.chain.invoke(
      {
        "query": HumanMessage(query),
        "memory": self.memory
      }
    )
    self.memory.append(HumanMessage(content=query))
    self.memory.append(AIMessage(content=response))
    
    return response

In [ ]:
llm = Ollama(model=GEMMA)
llm("¿Qué es la Inteligencia Artificial?")

In [ ]:
class BaseResponse:
  """
  Clase base para manejar respuestas del sistema. 
  Permite almacenar información de éxito/error y datos adicionales
  """
  def __init__(self, success:bool, agent:str, result:Any=None, message:str="", **kwargs:Dict[str,Any]) -> None:
    """Inicializa la respuesta base

    Args:
      success (bool): indica si la operación fue exitosa.
      agent (str): nombre del agente que ejecutó la operación.
      result (Any): resultado de la operación (puede ser cualquier tipo)
      message (str): descripción de la operación realizada. Defaults to "".
    """
    self.success = success
    self.agent   = agent 
    self.result  = result
    self.message = message 
    self.extra_data = kwargs

  def get_info(self) -> Dict[str,Any]:
    """Obtiene la información de la respuesta

    Returns:
      Dict[str,Any]: Diccionario con información básica de la respuesta
    """
    info = {
      "success": self.success,
      "agent": self.agent,
      "result": self.result,
      "message": self.message
    }
    info.update(self.extra_data)
    return info
  
  def get_data(self, key:str, default:Any=None) -> Any:
    """Obtiene un dato específico de los datos extra.

    Args:
      key (str): clave del dato a obtener
      default (Any): valor por defecto si la clave no existe. Defaults to None.

    Returns:
      Any: valor del dato solicitado o el valor por defecto
    """
    return self.extra_data.get(key, default)
  
  def __str__(self) -> str:
    "Representación en string de la información"
    status = "SUCCESS" if self.success else "ERROR"
    return f"[{status}] {self.agent}: {self.message}"
  
  def __repr__(self) -> str:
    "Representación detallada de la respuesta"
    return f"BaseResponse(success={self.success}, agent='{self.agent}', message='{self.message}')"

In [ ]:
class TaskType(Enum):
  TEXT_GENERATION = "generación de texto"
  DATASET_SEARCH = "consulta a dataset"
  MATH_OPERATION = "cálculo matemático"
  DOCUMENT_ANALYSIS = "análisis de documentos" 
  CODE_GENERATION = "generación de código"

class TaskStatus(Enum):
  PENDING = "pendiente"
  COMPLETED = "completado"
  FAILED = "fallido"

@dataclass
class Task:
  task_id:str 
  task_type:TaskType
  description:str
  result:Optional[Any] = None
  assigned_agent:Optional[str] = None
  status:TaskStatus = TaskStatus.PENDING
  error_message:Optional[str] = None

In [ ]:
class BaseAgent(ABC):
  """
  Clase base abstracta para todos los agentes del sistema.
  Define la interfaz común que deben implementar todos los agentes.
  """
  
  def __init__(self, name:str, description:str, capabilities:List[TaskType]) -> None:
    self.name:str = name 
    self.description:str = description
    self.capabilities:List[TaskType] = capabilities
  
  @abstractmethod
  def process(self, queries:List[str], results:List[Dict] = [], **kwargs:Dict[str,Any]) -> BaseResponse:
    "Método abstracto que debe implementar cada agente específico"
    pass 
  
  def can_handle_task(self, task_type:TaskType) -> bool:
    "Verifica si el agente puede manejar un tipo específico de tarea"
    return task_type in self.capabilities
  
  def get_info(self) -> Dict[str,str]:
    "Retorna información básica del agente"
    return {
      "name": self.name, 
      "description": self.description
    }

## Ejemplos

### Agente Especializado en Analizar y Generar Texto

In [ ]:
class DocumentTextAgent(BaseAgent):
  "Agente especializado en generar texto basado en una lista de documentos"
  
  def __init__(self, llm: LLM, documents: List[str]):
    super().__init__(
      name="DocumentTextAgent", 
      description="Genera texto y responde preguntas basándose en una colección de documentos",
      capabilities=[
        TaskType.TEXT_GENERATION, 
        TaskType.DOCUMENT_ANALYSIS
      ]
    )
    self.llm: LLM = llm 
    self.documents: List[str] = documents 
  
  def process(self, queries:List[str], results:List[Dict] = [], **kwargs: Dict[str, Any]) -> BaseResponse:
    "Procesa una consulta basándose en los documentos disponibles"
    if not queries:
      return BaseResponse(
        success=False,
        agent=self.name,
        result=None,
        message="No se encuentran 'queries' en los parámetros"
      )
    
    answers = []
    for query in queries:
      answer = self._process_query(query=query)
      answers.append(answer)
    
    return BaseResponse(
      success=True, 
      agent=self.name,
      result=answers,
      message="Procesamiento de consultas completado exitosamente"
    )
  
  def _process_query(self, query: str) -> str:
    "Procesa una consulta individual usando los documentos disponibles"
    enhanced_query = f""" 
    {query}
    ---
    Documentos disponibles:
    {self.documents}
    """
    response = self.llm(query=enhanced_query)
    return response

In [ ]:
llm = Ollama(model=GEMMA)
agent = DocumentTextAgent(llm, documents=["..."])
response = agent.process(queries=["¿qué es la inteligencia artificial?","¿qué aplicaciones tiene?"])

if not response.success:
  raise Exception(str(response))

answers:List[str] = response.result
for item in answers:
  print(item)
  print("=============")

### Agente Especializado en Resolver Problemas Matemáticos Generando y Ejecutando Código Python

In [ ]:
prompt_MathPythonAgent = """ 
Eres un asistente especializado en resolver problemas matemáticos usando Python.

Tu tarea es:
1. Analizar el problema matemático presentado
2. Generar código Python que resuelva el problema
3. El código debe ser seguro y ejecutable
4. Incluye solo las operaciones matemáticas necesarias
5. Asigna el resultado final a una variable llamada 'resultado'

Ejemplo:
Pregunta: "¿Cuánto es 2 + 2?"
Código:
```python
resultado = 2 + 2
```

IMPORTANTE: Responde SOLO con el código Python, sin explicaciones adicionales.
"""

class MathPythonAgent(BaseAgent):
  "Agente especializado en resolver problemas matemáticos y ejecutando código Python"
  
  def __init__(self, llm: LLM, documents: List[str]):
    super().__init__(
      name="MathPythonAgent", 
      description="Resuelve problemas matemáticos generando y ejecutando código Python",
      capabilities=[
        TaskType.CODE_GENERATION, 
        TaskType.MATH_OPERATION
      ]
    )
    self.llm: LLM = llm 
    self.documents: List[str] = documents 
  
  def process(self, queries:List[str], results:List[Dict] = [], **kwargs: Dict[str, Any]) -> BaseResponse:
    "Procesa una consulta basándose en los documentos disponibles"
    if not queries:
      return BaseResponse(
        success=False,
        agent=self.name, 
        result=None,
        message="No se encuentran 'queries' en los parámetros"
      )
    
    answers = [ ]
    for query in queries:
      answer = self._process_query(query=query)
      answers.append(answer)
    
    return BaseResponse(
      success=True,
      agent=self.name,
      result=answers,
      message="Procesamiento de cálculos matemáticos completado"
    )
    
  def _process_query(self, query: str) -> str:
    "Procesa una consulta individual para generar código Python"
    enhanced_query = f""" 
    {prompt_MathPythonAgent}
    
    Problema: {query}
    
    Documentos de referencia:
    {self.documents}
    """
    response = self.llm(query=enhanced_query)
    return response

## Agent Engine

**Agent Engine**: Motor Central que gestiona múltiples agentes y decide cuál usar para cada consulta

In [ ]:
prompt_AgentEngine = """
Tienes que elegir el agente más apropiado para responder la siguiente consulta.
Clasificada como: {task_type}

Agentes disponibles:
{agents_info}

Consulta del usuario: "{query}"

Responde ÚNICAMENTE con el nombre exacto del agente más apropiado (sin explicaciones adicionales).
"""

class TaskManager:
  def __init__(self, llm: LLM):
    self.task_queue: List[Task] = []
    self.agents: Dict[str, BaseAgent] = {}
    self.completed_tasks: List[Task] = []
    self.failed_tasks: List[Task] = []
    self.task_counter: int = 0
    self.llm: LLM = llm  # LLM para selección de agentes
  
  def add_agent(self, agent: BaseAgent) -> BaseResponse:
    "Registra un nuevo agente en el sistema"
    try:
      self.agents[agent.name] = agent
      return BaseResponse(
        success=True,
        agent=agent.name,
        result=None,
        message="Agente registrado exitosamente"
      )
    except Exception as e:
      return BaseResponse(
        success=False,
        agent=agent.name,
        result=None,
        message=f"Error al registrar agente: {str(e)}"
      )
  
  def add_task(self, task:Task) -> None:
    "Añade una nueva tarea en el sistema"
    self.task_queue.append(task)
  
  def get_available_agents(self) -> Dict[str, Dict[str, Any]]:
    "Devuelve la información de todos los agentes disponibles"
    return {name: agent.get_info() for name, agent in self.agents.items()}
  
  def select_agent_for_task(self, task: Task) -> Optional[BaseAgent]:
    "Selecciona el agente más apropiado para una tarea específica"
    if not self.agents:
      return None
    
    # Filtrar agentes que pueden manejar el tipo de tarea
    capable_agents = [
      agent for agent in self.agents.values() 
      if agent.can_handle_task(task.type)
    ]
    
    if not capable_agents:
      return None
    
    if len(capable_agents) == 1:
      return capable_agents[0]
    
    # Si hay múltiples agentes capaces, usar LLM para seleccionar el mejor
    agents_info = "\n".join([
      f"- {agent.name}: {agent.description} (Capacidades: {[cap.value for cap in agent.capabilities]})"
      for agent in capable_agents
    ])
    
    selection_prompt = prompt_AgentEngine.format(
      agents_info=agents_info,
      query=task.description,
      task_type=task.type.value
    ) 
    
    try:
      response = self.llm(selection_prompt).strip()
      
      # Buscar el agente por nombre en la respuesta
      for agent in capable_agents:
        if agent.name in response:
          return agent
      
      # Si no encuentra coincidencia exacta, devolver el primer agente capaz
      return capable_agents[0]
      
    except Exception:
      # En caso de error, devolver el primer agente capaz
      return capable_agents[0]
  
  def assign_task_to_agent(self, task: Task, agent: BaseAgent) -> bool:
    "Asigna una tarea específica a un agente"
    try:
      task.assign_to_agent(agent.name)
      return True
    except Exception:
      return False
  
  def execute_task(self, task: Task, results:List[Dict]) -> BaseResponse:
    "Ejecuta una tarea usando el agente asignado"
    if not task.assigned_agent:
      return BaseResponse(
        success=False,
        agent="TaskManager",
        result=None,
        message="Tarea no tiene agente asignado"
      )
    
    agent:BaseAgent = self.agents.get(task.assigned_agent)
    if not agent:
      return BaseResponse(
        success=False,
        agent="TaskManager",
        result=None,
        message=f"Agente {task.assigned_agent} no encontrado"
      )
    
    try:
      # Ejecutar la tarea con el agente
      response = agent.process(
        queries=[task.description],
        results=results
      )
      
      if response.success:
        task.mark_as_completed(response.result)
        self.completed_tasks.append(task)
      else:
        task.mark_as_failed(response.message)
        self.failed_tasks.append(task)
      
      return response
      
    except Exception as e:
      task.mark_as_failed(str(e))
      self.failed_tasks.append(task)
      return BaseResponse(
        success=False,
        agent=task.assigned_agent,
        result=None,
        message=f"Error al ejecutar tarea: {str(e)}"
      )
  
  def get_pending_tasks(self) -> List[Task]:
    "Obtiene todas las tareas pendientes"
    return [task for task in self.task_queue if task.status == TaskStatus.PENDING]
  
  def get_task_status(self) -> Dict[str, int]:
    "Obtiene un resumen del estado de las tareas"
    all_tasks = self.task_queue + self.completed_tasks + self.failed_tasks
    return {
      "pending": len([t for t in all_tasks if t.status == TaskStatus.PENDING]),
      "in_progress": len([t for t in all_tasks if t.status == TaskStatus.IN_PROGRESS]),
      "completed": len([t for t in all_tasks if t.status == TaskStatus.COMPLETED]),
      "failed": len([t for t in all_tasks if t.status == TaskStatus.FAILED])
    }


In [ ]:
DEFAULT_SYSTEM_PROMPT = """ 
Eres un asistente especializado capaz de responder preguntas sencillas
"""

DECOMPOSITION_SYSTEM_PROMPT = """ 
Eres un experto en descomposición de problemas. Tu tarea es tomar un problema complejo y dividirlo en subproblemas más pequeños y manejables.

Tener en cuenta:
- Cada subproblema debe ser específico y manejable
- Los subproblemas deben seguir un orden lógico
- Juntos, los subproblemas deben cubrir completamente el problema original

Formato de respuesta:
- Responde ÚNICAMENTE con una lista de subproblemas
- Un subproblema por línea
- Usa viñetas (`-`)

Ejemplo de salida:
- subproblema 1
- subproblema 2
- subproblema 3
"""

SELECT_PROMPT = """ 
Eres un experto en clasificar si una entrada de usuario es un subproblema de un problema mucho mayor. 

Formato de respuesta: 
- Responde ÚNICAMENTE con TRUE o FALSE
"""

CONSTRUCT_PROMPT = """ 
Eres un experto en crear preguntas. Tu tarea es tomar una entrada del usuario y generar a partir de esta una pregunta o problema.

Instrucciones:
- Generar la pregunta sencilla, corta y clara.
"""

def decompose_problem(problem:str, verbose:bool=False, k:int=3) -> List[str]:
  """Descompone un problema complejo en una lista de subproblemas más manejables.

  Args:
    problem (str): el problema principal a descomponer

  Returns:
    List[str]: lista de subproblemas que juntos resuelven el problema original
  """
  # crear una instancia temporal para descomposición
  decomposer = ChatOllama(model=GEMMA, temperature=0.2) # temperatura más baja para mayor consistencia
  decomposition_prompt = ChatPromptTemplate.from_messages([
    ('system', DECOMPOSITION_SYSTEM_PROMPT),
    ('human', f'Descompone el siguiente problema en subproblemas:\n\n{problem}')
  ])
  chain = decomposition_prompt | decomposer
  response = chain.invoke({"problem":problem}).content
  
  if verbose: print(response)
  
  subproblems = []
  lines = response.strip().split('\n')
  for line in lines:
    line = line.strip()
    
    # limpiar viñetas y numeración
    if line.startswith('- '):
      subproblem = line[2:].strip()
    elif line.startswith('• '):
      subproblem = line[2:].strip()
    elif re.match(r'^\d+\.\s*', line):
      subproblem = re.sub(r'^\d+\.\s*', '', line).strip()
    elif line and not line.startswith('#'):
      subproblem = line.strip()
    else:
      continue
    
    if subproblem and len(subproblem) > 5:   # filtrar lineas muy cortas
      subproblems.append(subproblem)
    
  if not subproblems:
    if k > 0:
      return decompose_problem(problem=problem, verbose=verbose, k=k-1)
    else:
      raise Exception("ERROR: no se pudo dividir el problema")
    
  return subproblems

def select_problem(problem:str, subproblem:str, verbose:bool=False) -> bool:
  selector = ChatOllama(model=GEMMA, temperature=0.2) # temperatura más baja para mayor consistencia
  selection_prompt = ChatPromptTemplate.from_messages([
    ('system', SELECT_PROMPT),
    ('human', '{problem}')
  ])
  chain = selection_prompt | selector
  response = chain.invoke({"problem":f"Problema original:\n\n{problem}\n\nEntrada de usuario:\n\n{subproblem}"}).content
  
  if verbose: print(response)
  
  lines = response.strip().split('\n')
  if len(lines) == 1: 
    try: 
      output = True if lines[0] == "TRUE" else False if lines[0] == "FALSE" else None
      if output == None:
        return False
        # raise Exception(f"ERROR al convertir a bool: {response}")
      return output
    except:
      return False
      # raise Exception(f"ERROR al convertir a bool: {response}")
  return False
  # raise Exception(f"ERROR al responder adecuadamente: {response}")

def construct_task(subproblem:str, verbose:bool=False) -> str:
  selector = ChatOllama(model=GEMMA, temperature=0.5) # temperatura más baja para mayor consistencia
  selection_prompt = ChatPromptTemplate.from_messages([
    ('system', CONSTRUCT_PROMPT),
    ('human', 'Entrada del usuario: {problem}')
  ])
  chain = selection_prompt | selector
  response = chain.invoke({"problem":subproblem}).content
  
  if verbose: print(response)
  
  return response

In [ ]:
CLASSIFICATION_PROMPT = """ 
Analiza la siguiente descripción de tarea y determina cuál de estos tipos de tarea es el más adecuado:

TIPOS DE TAREA DISPONIBLES:
1. TEXT_GENERATION (generación de texto) - Para crear contenido textual, escribir artículos, resúmenes, emails, etc.
2. DATASET_SEARCH (consulta a dataset) - Para buscar información específica en datasets, consultas SQL, filtros de datos, etc.
3. MATH_OPERATION (cálculo matemático) - Para realizar cálculos, operaciones matemáticas, análisis numérico, etc.
4. DOCUMENT_ANALYSIS (análisis de documentos) - Para analizar, extraer información o procesar documentos existentes
5. CODE_GENERATION (generación de código) - Para escribir código, programar funciones, crear scripts, etc.

DESCRIPCIÓN DE LA TAREA:
"{description}"

INSTRUCCIONES:
- Responde ÚNICAMENTE con el nombre del tipo de tarea (por ejemplo: TEXT_GENERATION)
- No incluyas explicaciones adicionales
- Si hay ambigüedad, elige el tipo más específico y relevante
- Considera las palabras clave y el contexto de la descripción

RESPUESTA:
"""

def get_task_type_by_description(description:str) -> TaskType:
  "Clasifica el tipo de task según los tipos de task disponibles"
  llm = Ollama(model=GEMMA, temperature=0.3)
  response = llm(CLASSIFICATION_PROMPT.format(description=description)).strip().upper()
  task_type_mapping: Dict[str, TaskType] = {
    "TEXT_GENERATION": TaskType.TEXT_GENERATION,
    "DATASET_SEARCH": TaskType.DATASET_SEARCH,
    "MATH_OPERATION": TaskType.MATH_OPERATION,
    "DOCUMENT_ANALYSIS": TaskType.DOCUMENT_ANALYSIS,
    "CODE_GENERATION": TaskType.CODE_GENERATION
  }
  
  if response in task_type_mapping:
    return task_type_mapping[response]
  
  return _classify_by_keywords(description)



def _classify_by_keywords(description: str, llm_response: str = "") -> TaskType:
  """Función auxiliar que clasifica por palabras clave como fallback.
    
  Args:
    description (str): Descripción de la tarea
    llm_response (str): Respuesta del LLM (opcional)
        
  Returns:
    TaskType: Tipo de tarea clasificado
  """
    
  # Convertir a minúsculas para comparación
  desc_lower = description.lower()
  response_lower = llm_response.lower()
    
  # Palabras clave por categoría
  keywords: Dict[TaskType, List[str]] = {
    TaskType.CODE_GENERATION: [
      "código", "programar", "script", "función", "algoritmo", 
      "python", "implementar", "desarrollar", "crear función", "generar código"
    ],
    TaskType.MATH_OPERATION: [
      "calcular", "matemática", "operación", "suma", "resta",
      "multiplicación", "división", "estadística", "análisis numérico",
      "ecuación", "fórmula", "porcentaje", "promedio"
    ],
    TaskType.DATASET_SEARCH: [
      "dataset", "datos", "buscar en", "consultar", "filtrar",
      "base de datos", "csv", "tabla", "registros", "consulta sql",
      "encontrar información", "extraer datos"
    ],
    TaskType.DOCUMENT_ANALYSIS: [
      "analizar documento", "revisar documento", "extraer información",
      "procesar archivo", "leer documento", "analizar texto",
      "interpretar", "evaluar documento", "examinar"
    ],
    TaskType.TEXT_GENERATION: [
      "escribir", "generar texto", "redactar", "crear contenido",
      "artículo", "resumen", "email", "carta", "descripción",
      "narrativa", "historia", "blog", "contenido"
    ]
  }
  
  # Buscar en la respuesta del LLM primero
  for task_type, words in keywords.items():
    for word in words:
      if word in response_lower:
        return task_type
    
  # Buscar en la descripción original
  task_scores: Dict[TaskType, int] = {task_type: 0 for task_type in TaskType}
  
  for task_type, words in keywords.items():
    for word in words:
      if word in desc_lower:
        task_scores[task_type] += 1
    
  # Retornar el tipo con mayor puntuación
  if max(task_scores.values()) > 0:
    return max(task_scores, key=task_scores.get)
    
  # Default si no se encuentra coincidencia
  return TaskType.TEXT_GENERATION


In [ ]:


class AgentEngine:
  """
  Interfaz principal donde TaskManager y los agentes se comunican 
  para resolver problemas complejos
  """
  
  def __init__(self, llm: LLM):
    self.task_manager = TaskManager(llm)
    self.llm: LLM = llm
    self.execution_history: List[Dict[str, Any]] = []
  
  def register_agent(self, agent: BaseAgent) -> BaseResponse:
    """Registra un nuevo agente en el sistema"""
    return self.task_manager.add_agent(agent)
  
  def decompose_complex_problem(self, problem: str) -> List[Task]:
    "Descompone un problema complejo en tareas más pequeñas. Retorna una lista de Task de cada tarea que debe realizarse"
    
    subproblems = decompose_problem(problem=problem, verbose=False)
    print(subproblems)
    selections = [subproblem for subproblem in subproblems if select_problem(problem=problem, subproblem=subproblem, verbose=False)]
    print(selections)
    tasks_defs = [construct_task(select) for select in selections  ]
    print(tasks_defs)
    
    tasks:List[Task] = []
    for i,task_def in enumerate(tasks_defs):
      tasks.append(
        Task(
          task_id=i,
          task_type=get_task_type_by_description(task_def),
          description=task_def
        )
      )
    
    return tasks
    
  def run_cycle(self, complex_problem: str) -> BaseResponse:
    """Ejecuta un ciclo completo del sistema para resolver un problema complejo
    
    Args:
      complex_problem (str): Descripción del problema complejo a resolver
      
    Returns:
      BaseResponse: Respuesta con el resultado del procesamiento
    """
    try:
      # 1. descomponer el problema complejo en tareas
      tasks = self.decompose_complex_problem(complex_problem)
      
      # 2. crear tareas en el TaskManager
      created_tasks = []
      for task in tasks:
        self.task_manager.add_task(task)
        created_tasks.append(task)
      
      results = []
      for task in created_tasks:
        # seleccionar agente apropiado
        selected_agent = self.task_manager.select_agent_for_task(task)
        
        if not selected_agent:
          task.mark_as_failed("No se encontró agente apropiado")
          continue
        
        # asignar tarea al agente
        self.task_manager.assign_task_to_agent(task, selected_agent)
        
        # ejecutar tarea
        execution_result = self.task_manager.execute_task(task, results)
        results.append({
          "task_id": task.id,
          "agent": selected_agent.name,
          "result": execution_result.result,
          "success": execution_result.success,
          "message": execution_result.message
        })
        
        #TODO actualizar tareas a partir de los resultados de la tarea ejecutada
      
      # 4. Consolidar resultados
      successful_results = [r for r in results if r["success"]]
      failed_results = [r for r in results if not r["success"]]
      
      # 5. Registrar en historial
      execution_record = {
        "problem": complex_problem,
        "tasks_created": len(created_tasks),
        "tasks_completed": len(successful_results),
        "tasks_failed": len(failed_results),
        "results": results,
        "task_status": self.task_manager.get_task_status()
      }
      self.execution_history.append(execution_record)
      
      # 6. Generar respuesta final
      if successful_results:
        final_result = {
          "problem": complex_problem,
          "solutions": [r["result"] for r in successful_results],
          "execution_summary": execution_record
        }
        
        return BaseResponse(
          success=True,
          agent="AgentEngine",
          result=final_result,
          message=f"Problema resuelto exitosamente. {len(successful_results)} tareas completadas."
        )
      else:
        return BaseResponse(
          success=False,
          agent="AgentEngine",
          result=execution_record,
          message="No se pudieron completar las tareas necesarias para resolver el problema"
        )
        
    except Exception as e:
      return BaseResponse(
        success=False,
        agent="AgentEngine",
        result=None,
        message=f"Error durante la ejecución del ciclo: {str(e)}"
      )
  
  def get_execution_history(self) -> List[Dict[str, Any]]:
    "Obtiene el historial de ejecuciones"
    return self.execution_history
  
  def get_system_status(self) -> Dict[str, Any]:
    "Obtiene el estado general del sistema"
    return {
      "registered_agents": len(self.task_manager.agents),
      "available_agents": list(self.task_manager.agents.keys()),
      "task_status": self.task_manager.get_task_status(),
      "execution_history_count": len(self.execution_history)
    }

In [ ]:
llm = Ollama(
  model=GEMMA,
  system_prompt="Eres un asistente inteligente que ayuda a resolver problemas complejos"
)

engine = AgentEngine(llm)

sample_documents = [
  """
# Generación de Números Aleatorios
  
La **generación de números aleatorios** se puede lograr a partir de la 
*simulación de variables aleatorias*. Esta simulación consiste en generar 
valores de una distribución específica (normal, exponencial, Poisson, 
etc.) y transformarlos en *números pseudoaleatorios uniformes* ($X \sim U(0,1)$). 
La simulación de un modelo basado en números aleatorios consiste en la construcción 
de un programa computacional que permite obtener los valores de salida para distintos
valores de las variables de entrada con el objetivo de obtener conclusiones del sistema 
que apoyen la toma de decisiones (explicar y/o predecir el comportamiento del sistema)
""",
""" 
# Aplicación de los Números Aleatorios
## Simulación y Modelación

Los números aleatorios permiten a los modelos matemáticos representar con mayor 
fidelidad la realidad, especialmente cuando se trata de simular entornos complejos 
donde intervienen numerosas variables impredecibles. Esto es particularmente relevante en: 

- Simulaciones meteorológicas y climáticas 
- Modelado de comportamientos humanos y sociales 
- Simulación de fenómenos físicos complejos 
- Pruebas de rendimiento de sistemas informáticos 

Estas aplicaciones aprovechan la capacidad de los números aleatorios para reflejar 
la incertidumbre inherente a los sistemas naturales y humanos, permitiendo la 
creación de modelos predictivos más robustos y realistas. 

## Juegos y Entretenimiento
Una de las aplicaciones más visibles y cotidianas es en la industria del entretenimiento: 
- Juegos de azar (loterías, casinos virtuales)
- Videojuegos (para generar eventos, comportamientos de NPCs, generación procedural de niveles)
- Aplicaciones de sorteos digitales

En estos contextos, la percepción de justicia y equilibrio está directamente vinculada a 
la calidad de la aleatoriedad generada
  """
]

agent1 = DocumentTextAgent(
  llm=llm,
  documents=sample_documents
)
agent2 = MathPythonAgent(
  llm=llm,
  documents=sample_documents
)

r_register1 = engine.register_agent(agent1)
r_register2 = engine.register_agent(agent2)
print(f"Resultado 1: {r_register1}")
print(f"Resultado 2: {r_register2}")

system_status = engine.get_system_status()
print(system_status)


**Ejemplo 1**: Descomposición de problemas complejos en tareas sencillas

In [ ]:
problem = """Haz un analisis completo de cómo funciona la Inteligencia Artificial
1. Definición de qué es la inteligencia artificial
2. Aplicación
3. Ejemplos 
4. Genera un código sencillo en Python de cómo trabaja la Inteligencia Artificial
"""

result = engine.decompose_complex_problem(problem=problem)
print(result)

In [ ]:
for item in result:
  print(item)
  print("==========")

**Ejemplo 3**: Problema complejo

In [ ]:
problem = """ 
Crea un reporte que combine
1. Análisis teórico de cómo funciona una variable aleatoria 
2. Genera un código en Python cómo se genera una variable aleatoria
"""
result = engine.run_cycle(problem)

if result.success:
  print(f"Message: {result.message}")
  
  solutions = result.result.get("solutions", [])
  for i, solution in enumerate(solutions, 1):
    print(f"Solucion {i}:")
    if isinstance(solution, list):
      for j, answer in enumerate(solution, 1):
        print(f" {j}. {answer}")
    else:
      print(f" {solution}")
else:
  print("ERROR")
  print(f"  {result.message}")